---
>「少なくとも2つの言語を理解しない限り、1つの言語を理解することはない」
 \
>（ジェフリー・ウィリアムズ）
---

# 自然言語処理（Natural Language Processing）

# 簡単な実装

ここでは、Transfomerを用いたBERTについて扱う
- 日本語は厄介なので、まずは英語を扱う
- Pytorch-Transformersを用いて学習済みBERTモデルによる簡単な事例を扱う

## Transformer
- 2017年に導入されたディープラーニングモデルの一種
  - 主に自然言語処理で利用されている
- RNNと同様自然言語などの時系列データ処理向けに設計されているが、再帰や畳み込みは利用していない
- Attention層のみで構築されている(後述)
- 翻訳やテキスト要約などの各種タスクに利用可能
- 並列化が容易で訓練時間を削減できる
- 「Attention is All You Need」という論文で著名になった

### Transformerの構造

<img src="http://class.west.sd.keio.ac.jp/dataai/text/transformer.png" width=300>

Seq2Seq同様EncoderとDecoderで構成

### Encoderの構造
1. Embedding層により入力文章をベクトルに圧縮、つまり分散表現に変換する
1. Positional Encoder層で文章内のどこにあるかという位置情報を加える
1. Multi-Head Attention層(後述)
1. normalization(正規化)によりデータの偏りを削減する
  - batch normalizationではなくlayer normalizationが行なわれる
1. Feed Forward層との組み合わせて処理され、実際のモデルでは6回繰り返される
  - 出力されたベクトルはDecoderに渡される
  - 特にPositionwise fully connected feed-forward networkと呼ばれる

- Multi-Head Attention層とFeed Forward層の組み合わせが6回繰り返される

以上で、Encoderが構成される

### Decoderの構造

1. Embedding層により入力文章をベクトルに圧縮(分散表現を獲得)
1. Positional Encoder層で位置情報を追加
1. Masked Multi-Head Attention層、先ほどと同様であるがAttention内のsoftmax関数を通す直前の値にマスキングが適用されている
  - 特定のkeyに対して、Attention weightを0にすることで入力した単語の先読みによる「カンニング」を防ぐ
  - 入力に予測すべき結果が入らないようにする
1. normalization（正規化）などで先ほどと同様
1. Multi-Head Attention層（Encoderの出力を入力として使用）
1. normalization（正規化）など
1. Positionwise fully connected feed-forward network(先ほどと同じ)
1. normalization（正規化）など
- 以上を6回繰り返す



### Transformerの構成要素

- Attention
  -「文章中のどの単語に注目すればよいかを表すスコア」のこと
  - Query、Key、Valueの3つのベクトルで求める
    - Query: Inputのうち「検索をかけたいもの」
    - Key: 検索対象とQueryの近さ、どれだけ似ているかを測る
    - Value: Keyに基づき、適切なValueを出力する
  - Self-Attention
    - 下図でInputとMemoryが同一のAttention
      - 文法の構造や、単語同士の関係性などを獲得するのに使用される
  - SourceTarget-Attention
    - 下図でInputとMemoryが異なるAttention
      - TransformerではDecoderで使用される
  - Multi-Head Attention
    - Attentionを複数並列して並べたもの(後述)
  - Masked Multi-Head Attention
    - Multi-Head Attentionにマスクをつけたもの
    - 特定の key に対してAttention weight を0にする
    - TransformerではDecoderで使われる
    - 入力した単語が先読みを防ぐために 情報をマスクで遮断する、言わば「カンニング」を防ぐ
  - Attentionは可視化できる
    - すでに示したが、attentionは可視化でき、どの単語に注目しているかを知ることができる
- Position-wise Fully-connected Feedforward Network
  - 2層からなる全結合ニューラルネットワーク
  - 単語の位置ごとに個別の順伝播ネットワークとなる
    - これにより他単語との影響関係を排除することができる
  - パラメータは全てのネットワークで共通
$FNN(x) = LeRU(xW_1+b_1)\cdot W_2+b_2$
- Positional Encoding ($PE$)
  - 「単語の位置」の情報をベクトルに加える
  - $pos$は位置を表し、$2i$および$2i+1$はEmbeddingの何番目の次元か、$d_{model}$が次元数を示す  
偶数番目：$PE_{(pos,2i)}=sin(pos/10000^{2i/d_{model}})$  
機数番目：$ PE_{(pos,2i+1)}=cos(pos/10000^{2i/d_{model}})$

<img src="http://class.west.sd.keio.ac.jp/dataai/text/attention2.png" width=600>

- 丸角(緑)がベクトル(テンソル)、四角角(青)が処理を表す
- **InputとMemoryはそれぞれ異なる埋め込みベクトルを表し、例えば2つの異なる文章を表す**
- Inputについて全結合層で各単語のQueryを作成する
- Memoryについても同様に全結合層でKeyを作成しQueryとの内積をとって関連度合い見る
  - 同じ向きを向いていれば掛け算となる
  - 垂直である、つまり関連しなければ0
  - この値を関連度(logit)とする
- logitにSoftmaxを適用して0から1の間に調整して出力、この結果が Attention weightとなる
  - メモリのどの単語に注意を払うかの重みづけ
  - QueryとKeyの関連が大きいとAttention weightが大きくなる
    - 正しくMemoryの単語に注意を向けるように,keyが正しくAttentionに向けられるように学習される
- Memoryから全結合層を経て、Memoryの各単語に対する埋め込みベクトルであるValueを算出する
  - ValueとAttenthion weightとの内積を求める
    - Attention weightに従ってValueを選択することを意味する
- 最後に全結合層を挟んで出力を得る




### InputとMemory

<img src="http://class.west.sd.keio.ac.jp/dataai/text/input-memory.png" width=400>

各文章は分かち書きされIDで表現された後、Embeddingにより埋め込みベクトルに変換される

### Attention Weightの算出

<img src="http://class.west.sd.keio.ac.jp/dataai/text/attention-weight.png" width=600>

QueryとKeyの内積を算出してInputとMemoryの各単語の関連度であるlogitを算出、Softmaxを用いてAttention weightとする
- Memoryのどの単語に注意を払うかの重み付け

例えば、Inputのスポーツという単語に対して、Memoryの「野球」 「が」 「得意」の各単語について正しく注意を向けるように学習する
- ここでは野球が高い値になるようになる

### valueとの内積

<img src="http://class.west.sd.keio.ac.jp/dataai/text/value-naiseki.png" width=600>

この内積は、value、ここでは「野球」「が」「得意」の各単語のValueとAttention weightを掛け合わせて総和を計算することになる

最も注目するべきvalueの値が算出されているといえるが、他の単語との関連性も考慮した値となっている

### Multi-Head Attention

<img src="http://class.west.sd.keio.ac.jp/dataai/text/mhattention.jpg" width=200>

- Attentionを並列させた構造を持つ
- それぞれのAttentionをHeadと呼ぶ
- 「Attention Is All You Need」ではMulti-Head化により性能が向上するとされている
  - アンサンブル学習に近い
  - Dropoutも毎回異なるネットワークを使っており、アンサンブルに通じるところがある

## BERT

BERT(Birdirectional Encoder Representation from Transformers)の略で、2018年の後半にGoogleが発表
- 自然言語処理のための新たなディープラーニングモデル
- Transformerをベースとしている
- 様々な自然言語処理タスクでファインチューニングが可能
  - 訓練済みモデルを与えられたタスクに合わせて調整可能- 従来の自然言語処理タスクよりも汎用性が高い

「BERT: Pre-training of Deep Bidirectional Transformers for  Language Understanding」という論文で登場



### BERTにおける学習

<img src="http://class.west.sd.keio.ac.jp/dataai/text/bert.png" width=600>

- 左側が事前学習モデル
  - ラベルのない、つまり正解のない文章のペアを渡す(Unlabeled Sentence A and B Pair)
    - Maskedとあるように文章の一部が隠されている
  - 各種出力がある
    - NSP = Next Sentence Predictionの略で次の文章を予測する時に使う
    - Mask LM = 文章の穴埋め補間に使う
   - BERTの学習には多大な計算コストが伴うため、事前学習モデルを利用することが多い
- Fine-Tuningは計算コストが比較的小さいので比較的手軽に実行可能
  - SQuADタスク
    - 「Stanford Question Answering Dataset」の略
    - 言語処理の精度を測るベンチマークであり約10万個の質問(Question)と回答(Paragraph)とのペアで構成される
    - このデータセットを用いてファイチューニングすることが行なわれている
    - 後述するNext Sequence Predictionにも関連するが、この質問と回答のセットを正しく選ぶことができるかどうかという問いに対し、BERTは人間よりも正答率を上げることができる
    - 画像分野では人間よりも正確に画像から病巣を見つけるなどできるようになったが、文章処理においても人間よりも正確に判断ができるようになったことで話題となった
  - NERタスク
    - 固有表現抽出で、文字列中に現れる（人名、組織名、日付などのような）様々な種類の固有表現に目印をつける
  - MNLIタスク
    - 自然言語推論のコーパス

- 事前学習
  - Transformerが、文章から文脈を双方向(Bidirectional)に学習
    - Masked Language ModelおよびNext Sentence Predictionによる双方向学習を用いる
- ファインチューニング
  - 事前学習により得られたパラメータを初期値として、ラベル付き(正解付き)のデータでファインチューニングを行う

日本語のBERT事前学習モデルが存在する
- 京都大学 黒橋・褚・村脇研究室
- http://nlp.ist.i.kyoto-u.ac.jp/index.php?BERT
- とにかくでかい


### BERTのモデル

<img src="http://class.west.sd.keio.ac.jp/dataai/text/bert2.png" width=300>

BERTに埋め込みベクトルEn(単語の分散表現)を入力する
- N個の単語が並んでいる文章とする
- TrmつまりTransformerに入力する
- 文章の単語の関連が、前後両方のリンクを伴うため、Bidirectional(双方向)とネーミングされている


### BERTの入力

<img src="http://class.west.sd.keio.ac.jp/dataai/text/bert3.png" width=600>

BERTの入力は文章の埋め込みベクトルである
- まず「my dog is cute. He likes play ##ing .」という文章を入力することを考える
  - 実際には文章の開始を意味する[CLS]、文章の区切りを表す[SEP]=セパレータを用いて表現される
- 各単語をEmbeddingする、つまり分散表現としての埋め込みベクトルに変換する
  - Segmentつまり属する文章を表現する
  - つまり、$E_A$の文章と$E_B$の文章の2つの文章のどちらに各単語が属するかを表す
- Position Embeddingsを加える
  - 文章のどの位置にあるかを表すベクトルである
  - ここでは、0から10の11単語存在することを表している


### BERTの学習

- 事前学習
  - Transformerが、文章から文脈を双方向(Bidirectional)に学習する
  - ここではMasked Language ModelおよびNext Sentence Predictionによる双方向学習を行う

- ファインチューニング
  - 事前学習により得られたパラメータを初期値として、ラベル付きのデータで ファインチューニングを行う


### Masked Language Model

文章から特定の単語を15％ランダムに選び、[MASK]トークンに置き換える
- 例: my dog is hairy → my dog is [MASK]
- [MASK]の単語を、前後の文脈から予測する


### Next Sentence Prediction
- 2つの文章に関係があるかどうかを判定する
  - 後ろの文章を50%の確率で無関係な文章に置き換える
  - 後ろの文章が意味的に適切であればIsNext、そうでなければNotNextと判定
- 例：[CLS] the man went to [MASK] store [SEP] / he bought a gallon [MASK] milk [SEP]
  - CLSは文章開始、SEPは文章の区切りや終わり
  - 判定：IsNext、つまり2つの文章は自然な流れである
- 例：[CLS] the man went to [MASK] store [SEP] / penguin [MASK] are flight #less birds  [SEP]
  - 判定：NotNext、つまり2つの文章は不自然な流れである


## Transformers

Transformers自然言語処理ライブラリでBERTが利用できる
  - 米国のHugging Face社が提供
  - 分類、情報抽出、質問回答、要約、翻訳、テキスト生成などのに利用できる事前学習モデルを100以上の言語で提供、話題のGPT-2もある
  - 最先端の自然言語処理技術が簡単に使用可能
  - PyTorchに対応
  - https://huggingface.co/transformers/

構成クラス
- model classes
  - 事前学習済みのパラメータを持つモデルのクラス
- configuration classes
  - ハイパーパラメータとして中間層のニューロンの数や層の構造などモデルの設定を行うためのクラス
tokenizer classes
  - 語彙の保持、形態素解析などに関連するクラス


### TransformersにおけるBERTモデル
- BertForPreTraining
これを継承して以下のモデルがある
- BertModel 特定タスクを想定しない汎用BERT
- BertForMaskedLM 単語の一部にマスクをかけてそれを予測
- BertForNextSentencePrediction ある文書の次の文章が適切か判別
- BertForSequenceClassification 文章を分類
- BertForMultipleChoice 文章を分類
- BertForTokenClassification 単語の分類
- BertForQuestionAnswering 質問と答えのペアを扱う

# PyTorch-Transformersの基本的な使い方


## ライブラリのインストール
PyTorch-Transformersおよび必要なライブラリをインストールする

In [ ]:
!pip install folium
!pip install urllib3
!pip install pytorch-transformers

## PyTorch-Transformersのモデル
PyTorch-Transformersには、様々な訓練済みのモデルを扱うクラスが用意されている

ここでは、文章の一部をMaskする問題である`BertForMaskedLM`のモデルを設定する
- 詳細はこちらhttps://huggingface.co/transformers/model_doc/bert.html#bertformaskedlm  

`BertForMaskedLM`はベースとなるモデルであり、`PreTrainedModel`を継承している
- 詳細はこちらhttps://huggingface.co/transformers/main_classes/model.html#transformers.PreTrainedModel  

さらに、`BertForMaskedLM`は`nn.Module`クラスも継承しており、今までと同様にPyTorchのモデルとして利用できる！

In [ ]:
import torch
from pytorch_transformers import BertForMaskedLM

msk_model = BertForMaskedLM.from_pretrained('bert-base-uncased')  # 訓練済みパラメータの読み込み
print(msk_model)

中身をみてみよう
- BertForMaskedLMとある
- BertModelの記述があり、word_embeddingsという単語をベクトルに変換する(分散表現化)
- BertEncoderで入力の特徴を抽出する
  - この中に0から11の12のBertレイヤがある(でかい)
  - Attention(特徴量を抽出)、Intermediate(特徴量の拡張)、Output(整形)がある
- BertPooler 出力をタスクに合わせて調整する
ここまでは全てのタスクで共通
- BertOnlyMLMHead 今回のタスクに特化している部分
  - 最終的に単語の数である30522クラスに分類する問題であるとわかる

<img src="http://class.west.sd.keio.ac.jp/dataai/text/pytorch-transbert.png" width=600>

- BertEmbeddings
  - 入力を埋め込みベクトルに変換
  - word_embeddings, position_embeddings, token_type_embeddings、これら3つの埋め込みベクトルを足し合わせる

- BertAttention
  - 入力を埋め込みベクトルに変換
  - word_embeddings, position_embeddings, token_type_embeddings、これら3つの埋め込みベクトルを足し合わせる
  - https://git.io/JkniK で確認するとよい(github.com専用のURL短縮サービスを利用)
BertAttention
  - BertSelfAttention、BertSelfOutputによりSelf-Attentionを実装
  - Maskの実装
  - 以下を利用して確認するとよい  
https://git.io/JknTg （BertAttention）  
https://git.io/JknTi （BertSelfAttention）  
https://git.io/JknXl （BertSelfOutput）

- BertIntermediateおよびBertOutput
  - Positionwise fully connected feed-forward networkを実装
  - 以下を利用して確認するとよい  
https://git.io/Jkn1u （BertIntermediate）  
https://git.io/Jkn1r （BertOutput）

- BertPooler
  - 最初のトークン（単語）を取得し全結合層、活性化関数で処理
  - Next Sentence Predictionなどで使用
  - https://git.io/JknUw で確認するとよい


さらに文章を分類するモデルである`BertForSequenceClassification`を設定する
- 詳細はこちらhttps://huggingface.co/transformers/model_doc/bert.**html**#bertforsequenceclassification  

In [ ]:
from pytorch_transformers import BertForSequenceClassification

sc_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')  # 訓練済みパラメータの読み込み
print(sc_model)

中身を見てみよう

- 最初は先ほどと全く同じである
- 違いは最後のdropoutと全結合層となる
- `out_features=2`とあるため、文章を2クラスに分類する問題であることが分かる


# BERTの設定
`BertConfig`クラスによりモデルの設定が可能である  

In [ ]:
from pytorch_transformers import BertConfig

config = BertConfig.from_pretrained("bert-base-uncased")
print(config) 

以上が初期設定である
- `hidden_size` 隠れ層のノードの数
- `vocab_size` 単語数
- `type_vocab_size` セグメントの数、2なので2つの文章を扱う
- `intermediate_size` BERT intermediate layerのノード数

## Tokenizer
`BertTokenizer`クラスを使って、訓練済みのデータに基づく形態素解析が可能となる

In [ ]:
from pytorch_transformers import BertTokenizer

text = "I have a pen. I have an apple."

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
words = tokenizer.tokenize(text)
print(words)

では、BertForMaskedLM(文章におけるMASKされた単語の予測)とBertForNextSentencePrediction(ある文章の、次の文章が適切かどうかの判定)を実際に行う




## 文章の一部の予測
文章における一部の単語をMASKし、それをBERTのモデルを使って予測する

In [ ]:
text = "[CLS] I played baseball with my friends at school yesterday [SEP]"
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
words = tokenizer.tokenize(text)
print(words)

英語の文章であると、どうも答えがつまらない

文章の一部をMASKする
- MASKする場所は、様々に変更してみるとよいだろう

In [ ]:
msk_idx = 3  # ここでは3番目の単号であるbaseballをマスクする
words[msk_idx] = "[MASK]"  # baseballを[MASK]に置き換える
print(words)

単語を対応するインデックスに変換する

In [ ]:
word_ids = tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
word_tensor = torch.tensor([word_ids])  # テンソルに変換
print(word_tensor)

BERTのモデルを使って予測する
- from_pretrainedで学習済みモデルを利用する
- 学習は行わないため、evalを指定する

In [ ]:
msk_model = BertForMaskedLM.from_pretrained("bert-base-uncased")
msk_model.cuda()  # GPU対応
msk_model.eval()

x = word_tensor.cuda()  # GPU対応
y = msk_model(x)  # 予測(順伝搬)
result = y[0]  # tuppleなので要素を取り出す
print(result.size())  # 結果の形状

resultの形状を確認する
- 文章が11個の単語で構成されており、30522個の単語それぞれの適切度合いが含まれていることがわかる

ここから、値の大きい5個を取り出す
- 結果がバッチ対応であるため0とする
- 最も大きな値そのものが`_`の部分に入るが、利用しないためその値を捨てられる

In [ ]:
_, max_ids = torch.topk(result[0][msk_idx], k=5)  # 最も大きい5つの値
result_words = tokenizer.convert_ids_to_tokens(max_ids.tolist())  # インデックスを単語に変換
print(result_words)

きちんと「play」できるものが選択されている
- pianoなど楽器はtheが伴うので含まれていない！流石！

## 文章が連続しているかどうかの判定
BERTのモデルを使って、2つの文章が連続しているかどうかを判定(確率表示)する関数`show_continuity`で2つの文章の連続性を判定する

In [ ]:
from pytorch_transformers import BertForNextSentencePrediction

def show_continuity(text, seg_ids):
    words = tokenizer.tokenize(text)
    word_ids = tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
    word_tensor = torch.tensor([word_ids])  # 文章そのもの、テンソルに変換
    seg_tensor = torch.tensor([seg_ids])  # どこで前と後ろの文章が区切られるか、同様にテンソルに変換

    nsp_model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
    nsp_model.cuda()  # GPU対応
    nsp_model.eval()

    x = word_tensor.cuda()  # 文章そのもの、GPU対応
    s = seg_tensor.cuda()  # 分割場所、GPU対応

    y = nsp_model(x, s)  # 予測
    result = torch.softmax(y[0], dim=1)  # 確率表現に変換
    print(result)  # Softmaxで確率になっているのでそれを表示
    print(str(result[0][0].item()*100) + "%の確率で連続しています。")

`show_continuity`関数に、自然につながる2つの文章を与えると、当然ながら高い値が出る

In [ ]:
text = "[CLS] What is baseball ? [SEP] It is a game of hitting the ball with the bat [SEP]"
seg_ids = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1, 1]  # 0:前の文章の単語、1:後の文章の単語
show_continuity(text, seg_ids)

なお、いじめてみるとわかるが、それほど賢くはない

In [ ]:
text = "[CLS] What is baseball ? [SEP] It is a game of hitting the ball with the ball [SEP]"
seg_ids = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1, 1]  # 0:前の文章の単語、1:後の文章の単語
show_continuity(text, seg_ids)

`show_continuity`関数に、自然につながらない2つの文章を与えてみる

In [ ]:
text = "[CLS] What is baseball ? [SEP] This food is made with flour and milk [SEP]"
seg_ids = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]  # 0:前の文章の単語、1:後の文章の単語
show_continuity(text, seg_ids)

値が小さくなるため、関連していないとわかる

# ファインチューニング

事前学習済みのモデルに、追加で訓練する

## ライブラリのインストール

PyTorch-Transformaersの他、Transformersというライブラリも存在する
- Transformersは、PyTorch-Transformersの上位互換ライブラリとして構築されている(オンラインドキュメントで移行手法が提示されている)
- ここではTransformarsを利用する

https://huggingface.co/transformers/index.html

In [ ]:
!pip install transformers

## モデルの読み込み
`BertForSequenceClassification`により、事前学習済みのモデルを読み込みます。

In [ ]:
from transformers import BertForSequenceClassification

sc_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", return_dict=True)
print(sc_model.state_dict().keys())

## 最適化アルゴリズム
最適化アルゴリズムにAdamWを利用する
- オリジナルのAdamに対し、重みの減衰式が更新されている

In [ ]:
from transformers import AdamW

optimizer = AdamW(sc_model.parameters(), lr=1e-5)

## Tokenizerの設定

- BertTokenizerにより文章を単語に分割、idに変換
- BertForSequenceClassificationのモデルを訓練する際には入力の他にAttention maskが必要となるが、BertTokenizerによりmask情報も得ることができる
  - 未来の情報を与えないようにする(チートしないようにする)
- return_tensorsを"pt"とすることで、PyTorch Tensor型を扱うようにする

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
sentences = ["I love baseball.", "I hate baseball."]
tokenized = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)
print(tokenized)

x = tokenized["input_ids"]  # 単語のID(input_ids)を取り出してxすなわち入力とする
attention_mask = tokenized["attention_mask"]  # 同様にattention maskも取り出すことができる

結果を見てみよう

- 各単語のIDが次のように得られる `'input_ids': tensor([[ 101, 1045, 2293, 3598, 1012,  102], [ 101, 1045, 5223, 3598, 1012,  102]])`
  - 2つ文章があるので別の次元で格納されている

- 文章は2つ含まれているが、ここはまとめて一つの文章ID0として扱われている `'token_type_ids': tensor([[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])`

- Attention Maskは、`'attention_mask': tensor([[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]])}`であり、すべて1、つまりマスクしないことを意味している

## ファインチューニングの実施

転移学習(Transfer Learning)については既に学んだため、ここでは省略する

たった2つの文章で訓練するため、全く効果は期待できないが、まずは手法を理解する



In [ ]:
import torch
from torch.nn import functional as F
import matplotlib.pyplot as plt

sc_model.train()  # 訓練モードにする
t = torch.tensor([1,0])  # 文章の分類(loveの文章が1でポジティブ、hateの文章が0でネガティブ)
weight_record = []  # 重みを記録
loss_record = []  # ロスを記録
for i in range(100):
  y = sc_model(x, attention_mask=attention_mask)
  loss = F.cross_entropy(y.logits, t)
  loss.backward()
  optimizer.step()
  weight = sc_model.state_dict()["bert.encoder.layer.11.output.dense.weight"][0][0].item()
  weight_record.append(weight)
  loss_record.append(loss.detach().numpy()) # とりあえずメモリ共有で、変更する場合は .copy()とする
  if i%10 == 0:
    print(i, "-> Loss:", loss.detach().numpy(), " Weight:", weight)

まずLossを見ると、学習が進んで値が小さくなることがわかる

In [ ]:
plt.plot(range(len(loss_record)), loss_record)
plt.show()

次にWeightを見ると、値がどんどん変化して更新されていることがわかる
- 追加の訓練により、重みが調整されていく様子が確認できる

In [ ]:
plt.plot(range(len(weight_record)), weight_record)
plt.show()

## ファインチューニングによる感情分析
ファインチューニングを活用し、文章の好悪感情を判別できるようにモデルを訓練してみよう

## ライブラリのインストール

nlpライブラリをインストールする
- 自然言語処理ライブラリで、付随データセットを利用させてもらうために導入する  

In [ ]:
!pip install nlp

## モデルとTokenizerの読み込み
事前学習済みのモデルと、これと紐づいたTokenizerを読み込む
- クラス分類を行うためBertForSequenceClassificationを利用する
- 高速な字句解析を行うBertTokenizerFastを利用する
- いつもと同様、bert-base-uncased 事前学習モデルを読み込む

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizerFast

sc_model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
sc_model.cuda()
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

## データセットの読み込み

nlpライブラリに含まれるIMDbデータセットを利用する
- IMDbデータセットは、ポジティブかネガティブの好悪感情を表すラベルが付与された25000の映画レビューコメントデータセット
- 好意的なレビューは1、否定的なレビューは0が振られている
- 感情分析用では鉄板のデータセット

https://www.imdb.com/interfaces/

In [ ]:
from nlp import load_dataset

train_data, test_data = load_dataset("imdb", split=["train", "test"]) # 訓練用と検証用データに分けて読み込む

試しにデータを表示されてみる
- 英語です、がっかりしましたか？

In [ ]:
print(train_data["label"][0], train_data["text"][0])  # 好意的なコメントの例
print(train_data["label"][20000], train_data["text"][20000])  # 否定的なコメントの例

DeepLで訳してみると次のような感じです

> 1 ブロムウェル・ハイ」は、カートゥーン・コメディです。ブロムウェル・ハイ』は、『ティーチャーズ』のような学校生活を描いた番組と同時期に放送されていました。私の35年間の教師生活を振り返ると、「ブロムウェル・ハイ」の風刺は「ティーチャーズ」よりもはるかに現実に近いものだと思います。経済的に生き残るために奔走する姿、哀れな教師たちの虚勢を見抜く洞察力のある生徒たち、そしてすべての状況の情けなさは、私が知っている学校とその生徒たちを思い出させてくれます。生徒が何度も学校を燃やそうとしたエピソードを見たとき、すぐに ......... .......... のことを思い出しました。高いですね。古典的なセリフです。検閲官：あなた方の先生の一人をクビにするために来ました。生徒：Bromwell Highへようこそ。私と同年代の大人の多くは、「ブロムウェルハイ」を奇想天外なものだと思っているのではないでしょうか。そうでないのが残念です。

> 0 この映画は努力していますが、1960年代のテレビシリーズの面白さが完全に欠けています。私は17歳ですが、ずいぶん前にYouTubeでこのシリーズを見たことがあり、楽しくて仕方がありませんでした。特殊効果は標準的ではなく、平板なカメラワークによって助けられていませんでした。また、「ホームアローン4」、「帽子をかぶった猫」、「きかんしゃトーマス」、「アダムス・ファミリー・リユニオン」などの作品があります。さて、ストーリーのアイデアは良かったのですが、残念ながら出来が悪く、早々に力尽きてしまったので、正直、家族で楽しめる作品ではないと思います。また、ウェイン・ナイトが気合を入れて演じたにもかかわらず、しゃべるスーツにも腹が立ちました。しかし、この映画で最も腹が立ったのは、クリストファー・ロイド、ジェフ・ダニエルズ、ダリル・ハンナという才能ある俳優を無駄にしてしまったことです。ジェフ・ダニエルズはこれまでも良い演技をしてきましたが、彼は何をすべきかわからないようでしたし、エリザベス・ハーリーのキャラクターも残念ながら役立たずでした。ダリル・ハンナは素敵な女優だが、一般的には無視されており、私は彼女が愛の対象になるというアイデアが好きだったが、残念ながら彼女の姿はほとんど見られない。（モンスターの攻撃は、子供たちを魅了するというよりも、怖がらせる可能性が高いのは言うまでもない）同様に、ウォレス・ショーンもある種の政府の工作員として登場する。        1/10 ベサニー・コックス

mapメソッドを利用して各データに前処理を施す
- ここではtokenizeを定義し、このtokenizeを全データに施す
- tokenizeは読み込んだIMDbのデータをTokenizerで処理し、語句IDに変換する関数である
- バッチサイズはデータ全体、つまり全データに対して一気に処理している(順番に取り出して何かするのではないため、これでよい)
- "input_ids", "attention_mask", "label"の順番にデータを並べて、PyTorchで利用できるようにPyTorchのDataLoaderと同様の形で出力させる

In [ ]:
def tokenize(batch):
  return tokenizer(batch["text"], padding=True, truncation=True)

train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
train_data.set_format("torch", columns=["input_ids", "attention_mask", "label"])

test_data = test_data.map(tokenize, batched=True, batch_size=len(train_data))
test_data.set_format("torch", columns=["input_ids", "attention_mask", "label"])

## 評価用の関数
`sklearn.metrics`を用いてモデル評価のための関数を定義する
- バッチ対応されているので楽
- 使うのはaccuracy_score

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(result):
  labels = result.label_ids  # こちらが正解
  preds = result.predictions.argmax(-1)  #  予測値のうち値が最も大きい要素のインデックスを取り出す
  acc = accuracy_score(labels, preds)  #  両者を比較する
  return {
    "accuracy": acc,
  }

## Trainerの設定
Trainerクラス、およびTrainingArgumentsクラスを使用して、訓練を行うTrainerを設定する
- Trainerクラスを用いることで、それなりに面倒なファイチューニングを簡単に行うことができる
- TrainingArgumentsクラスで、ハイパーパラメータを適宜集約できる
  - 様々なファインチューニングで必要なパラメータ調整項目が設けられているので一度マニュアルを確認するとよい
  - 学習係数を500ステップまで値を上昇させ、その後weight_decayで下降に転じる手法をとる
    - 論文でもそのような方針が示されている
    - 複雑である場合、最初の一歩で大きく動かず、まずは確実に最適な方向に向いてから近づけるといった感覚で、これはポテンシャル場のイメージが浮かばないと、これでよいのかどうかはよくわからないところ(ということで論文で言われている通りにする)
  - train_batch_sizeのデフォルト値は16であるが、ColaboratoryのGPUをもってしても一部のGPUではメモリ不足となるため8などとする必要があるかもしれない
  - 16でおよそ30分弱程度、8で40分程度かかる
- ログをlogsフォルダに保存する
  - なお、TensorBoardに対応した形で保存してくれる(後で試そう)

Trainerを使って訓練する
- modelでモデルを指定、ここでは先に宣言したsc_model
- argsでハイパーパラメータを指定、ここではtraining_args
- compute_metricsで評価用関数を指定
- train_datasetで訓練用データを指定
- eval_datasetで評価用データを指定

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
  output_dir = "./results",
  num_train_epochs = 1,
  per_device_train_batch_size = 16,  # 大きい方が効率がよいがメモリ制約に引っかかりがちなので注意
  per_device_eval_batch_size = 32,
  per_gpu_train_batch_size = 8,
  warmup_steps = 500,  # 学習係数が0からこのステップ数で上昇
  weight_decay = 0.01,  # 重みの減衰率
  # evaluate_during_training = True,  # ここの記述はバージョンによっては必要ない
  logging_dir = "./logs",
)

trainer = Trainer(
  model = sc_model,
  args = training_args,
  compute_metrics = compute_metrics,
  train_dataset = train_data,
  eval_dataset = test_data
)

## モデルの訓練

設定に基づきモデルを訓練する
- さすがに専用の統合環境なので、実行時間表示などが美しく、終了予定時刻表示も備わっており素晴らしい
- 30分程必要となるので注意すること

In [ ]:
trainer.train()

## モデルの評価
Trainerの`evaluate()`メソッドによりモデルを評価する
- 今回は評価だけでも10分弱必要
- Accuracyは90%を簡単に超える値になるであろう
- 実はロスは頭打ちになっておらず、Accuarcyもサチュレーションを起こしてない！
  - まだまだ伸びるが、時間が…

In [ ]:
trainer.evaluate()

## TensorBoardによる結果の表示
TensorBoardを使ってlogsフォルダに格納された学習過程を表示する

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

# 日本語文章の分類

日本語のデータセットで学習したBERTモデルをファインチューニングし、ニュース分類を行う

Googleは2019年10月25日にBERTを検索エンジンに組み込むなど、広く応用されている
- 検索クエリに文章を入力した場合、より適切なページに誘導されるようになっている
- LaBSE: Language-agnostic BERT sentence embedding modelを利用
  - 109言語を事前学習させ学習データにない言語でも性能を発揮！
  - 複数の言語を同一空間の中に表現し、同一意味が近い位置になるように潜在空間を構成している
  - 知らない言語でも、その空間上でどこかにマップされれば、その付近の解釈であろうと判断する
- BERT採用事例をGoogleなどで検索すると、数多くの企業・サービスがBERTを利用していることがわかる
  - 特許検索、Q&A検索、金融、医療、サポート応答など文章検索に広く応用されている

## データセット

先にも使ったlivedoorニュースコーパスを用いた記事分類を行う
- 精度比較にも丁度良いであろう
- シートが分かれているので、残念ながらもう一度ダウンロードが必要

In [ ]:
import os
if not os.path.exists('text/topic-news/LICENSE.txt'):
  # ファイルが暗号化されているが、これはgoogle driveによるウィルス誤検出を回避するためである。
  #!wget "https://drive.google.com/uc?export=download&id=15EvNnKB6Y6-jGpo1q6N5BZ8SqMI-xzze" -O ldcc-20140209.zip
  !wget https://keio.box.com/shared/static/agjdm4m93o5lay6k0uy9wfadqi79hwpm -O ldcc-20140209.zip
if not os.path.exists('text'):
  !unzip -P dataai ldcc-20140209.zip

## 学習済みモデル
Pretrained Japanese BERT modelsを利用
- 東北大学、乾(いぬい)研究室が作成した日本語モデル
- https://github.com/cl-tohoku/bert-japanese


## ライブラリのインストール

いつものようにライブラリTransformers、およびnlpをインストール、その他livedoorのデータを利用するためのdatasets、関連するfugashi、ipadicを導入する
- fugashiは、MeCabのwrapperで、形態素解析を簡単に動かしたい場合に役に立つ
- ipadicは、IPA辞書（IPADIC）のことで、MeCab公式が推奨している辞書である
  - この辞書は形態素解析器ChaSen用辞書として作成されMeCab用に調整されている
- このあたりの精度の良い鉄板ライブラリの利用も、従来は苦労して導入していたが、今や簡単に導入できるようになった

In [ ]:
!pip install transformers
!pip install nlp
!pip install datasets
!pip install fugashi
!pip install ipadic

## データセットの読み込み

Googleドライブに保存されている、ニュースのデータセットを読み込む

In [ ]:
import glob  # ファイルの取得に使用
import os

path = "text/"  # フォルダの場所を指定

dir_files = os.listdir(path=path)
dirs = [f for f in dir_files if os.path.isdir(os.path.join(path, f))]
  # ディレクトリ一覧を取得
text_label_data = []  # 文章とラベルのセット
dir_count = 0  # ディレクトリ数のカウント
file_count= 0  # ファイル数のカウント

for i in range(len(dirs)):
  dir = dirs[i]
  files = glob.glob(path + dir + "/*.txt")
    # すべてのディレクトリにある.txtファイルの一覧をワイルドカードで取得
  dir_count += 1
  for file in files:
    if os.path.basename(file) == "LICENSE.txt":  # LICENSEファイルは無視
      continue
    with open(file, "r") as f:
      text = f.readlines()[3:]  # 先頭3行は日付やタイトルなので削除
      text = "".join(text)  # 配列になるのですべてを結合して一つの文章にする
      text = text.translate(str.maketrans({"\n":"", "\t":"", "\r":"", "\u3000":""}))
        # 改行やタブ、特殊文字を削除(便利) 
      text_label_data.append([text, i]) # 本文とラベル情報を合わせてtest_label_dataに追加

    file_count += 1
    print("\rfiles:" + str(file_count) + " dirs:" + str(dir_count), end="")

ファイルは7367個、9個のフォルダつまり分類がある

## データの保存
データを訓練データとテストデータに分割し、csvファイルとしてGoogle Driveに保存する

これらのファイルを別途取得していれば、この後のセル以降を実行することで学習が可能となる

In [ ]:
import csv
from sklearn.model_selection import train_test_split

news_train, news_test =  train_test_split(text_label_data, shuffle=True)
  # 訓練用とテスト用に分割
news_path = "text/"  # データを保存する場所(揃えてtextにしている)
with open(news_path+"news_train.csv", "w") as f: # 訓練データ
    writer = csv.writer(f)
    writer.writerows(news_train)
with open(news_path+"news_test.csv", "w") as f: # 検証データ
    writer = csv.writer(f)
    writer.writerows(news_test)

## モデルとTokenizerの読み込み
日本語の事前学習済みモデルと、これと紐づいたTokenizerを読み込む
- BertJapaneseTokenizerとして日本語対応分かち書きライブラリを利用する
  - `cl-tohoku/bert-base-japanese-whole-word-masking`と日本語対応を読み込む
  - 東北大乾研モデル
- BERTモデルも日本語対応版を利用する
  - `cl-tohoku/bert-base-japanese-whole-word-masking`
  - ラベル数は9である
  

In [ ]:
from transformers import BertForSequenceClassification, BertJapaneseTokenizer

sc_model = BertForSequenceClassification.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking", num_labels=9)
sc_model.cuda()
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

## データセットの読み込み
保存されたニュースのデータを読み込む
- 前回同様tokenize関数を定義し、これをmapしてデータを作成する
  - 文章の最大長は512であるが処理負荷を考え128に限定している(長くしてもよいだろう)
- load_datasetでは、csv形式と指定してファイルパスからデータを読み込む
  - カラム名はファイルを見ればわかるが、textとlabelとする
  - splitで訓練データと検証データに分けることができるが、すでに分割済みでありここでは訓練データを読み込むためtrainとだけ指定している
  - ここでも全体一気にmapする
  - set.formatでPyTorchを使うため'torch'と指定、カラムはinput_idsとlabelとする
- 同様に訓練データもロードする



In [ ]:
from datasets import load_dataset

def tokenize(batch):
  return tokenizer(batch["text"], padding=True, truncation=True, max_length=128)
    
news_path = "text/"
train_data = load_dataset("csv", data_files=news_path+"news_train.csv", column_names=["text", "label"], split="train")
train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
train_data.set_format("torch", columns=["input_ids", "label"])
test_data = load_dataset("csv", data_files=news_path+"news_test.csv", column_names=["text", "label"], split="train")
test_data = test_data.map(tokenize, batched=True, batch_size=len(test_data))
test_data.set_format("torch", columns=["input_ids", "label"])

## 評価用の関数
`sklearn.metrics`を使用し、モデルを評価するための関数を定義する
- 前回と同じ


In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(result):
  labels = result.label_ids
  preds = result.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
    "accuracy": acc,
  }

## Trainerの設定
Trainerクラス、およびTrainingArgumentsクラスを使用して、訓練を行うTrainerを設定する
- 前回と同じ

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
  output_dir = "./results",
  num_train_epochs = 2,
  per_device_train_batch_size = 16, # メモリが溢れる場合は8にしてみよう
  per_device_eval_batch_size = 32,
  warmup_steps = 500,  # 学習係数が0からこのステップ数で上昇
  weight_decay = 0.01,  # 重みの減衰率
  # evaluate_during_training = True,  # ここの記述はバージョンによっては必要ありません
  logging_dir = "./logs",
)

trainer = Trainer(
  model = sc_model,
  args = training_args,
  compute_metrics = compute_metrics,
  train_dataset = train_data,
  eval_dataset = test_data,
)

## モデルの訓練

設定に基づきファインチューニングを行う
- バッチサイズが16であれば今度は5分程度で終了する


In [ ]:
trainer.train()

## モデルの評価

Trainerの`evaluate()`メソッドによりモデルを評価する

In [ ]:
trainer.evaluate()

## TensorBoardによる結果の表示
TensorBoardを使ってlogsフォルダに格納された学習過程を表示する

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

## モデルの保存
訓練済みのモデルを保存する
- 一般的なPyTorchの保存は既に扱った
- 同様に保存して読み込んでみよう

In [ ]:
news_path = "text/"
sc_model.save_pretrained(news_path)
tokenizer.save_pretrained(news_path)

## モデルの読み込み
保存済みのモデルを読み込む

In [ ]:
loaded_model = BertForSequenceClassification.from_pretrained(news_path) 
loaded_model.cuda()
loaded_tokenizer = BertJapaneseTokenizer.from_pretrained(news_path)

## 日本語ニュースの分類

読み込んだモデルを使ってニュースを分類する
- 基本的には既に扱った通り

一つ記事を選択する

In [ ]:
import glob  # ファイルの取得に使用
import os
import torch

category = "movie-enter"  # 映画に関する記事を取り出す
sample_path = "text/"  # フォルダの場所を指定
files = glob.glob(sample_path + category + "/*.txt")  # ファイルの一覧
file = files[12]  # 適当なニュース(ここでは12番目)

dir_files = os.listdir(path=sample_path)
dirs = [f for f in dir_files if os.path.isdir(os.path.join(sample_path, f))]  # ディレクトリ一覧

with open(file, "r") as f:
  sample_text = f.readlines()[3:]
  sample_text = "".join(sample_text)
  sample_text = sample_text.translate(str.maketrans({"\n":"", "\t":"", "\r":"", "\u3000":""})) 

print(sample_text)

では、このテキストの記事を当ててみよう

In [ ]:
max_length = 512
words = loaded_tokenizer.tokenize(sample_text)
word_ids = loaded_tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
word_tensor = torch.tensor([word_ids[:max_length]])  # テンソルに変換

x = word_tensor.cuda()  # GPU対応
y = loaded_model(x)  # 予測
pred = y[0].argmax(-1)  # 最大値のインデックス
print("result:", dirs[pred])

他の記事についても試してみるとよい
- また、自分で勝手に記事を考え、分類させてみるのも面白いであろう

## BERTSUM

文章の要約を行うことが最近よく行われている
- これを行う専用のBERTが文章要約用のBERTSUMである
- BERTSUMは、文章の開始を意味するCLSを、文章の区切りにも挿入する
- 文章がどのセグメントに属するかを指定する際、偶数番目と奇数番目の文章でそれぞれ例えば$E_A$や$E_B$といった具合に異なるセグメントに所属させる
- Encoder-Decoderの形態を有し、Encoderは事前学習モデルを用い、Decoderはゼロから学習させる
- 英語はもちろんのこと、日本語の実装もgithubで公開されている
  - 英語 https://git.io/JMDSL
  - 日本語 https://git.io/JMDSG


# まとめ

今回は、自動翻訳や記事分類、BERTなど自然言語処理の基本について扱ったが、大量の優れた学習データと、さらに多いノード数を持つ構造を利用できれば、翻訳性能や分類性能を向上できることがわかるであろう
- 結局、最後はデータなのである
- さらに大量のデータを利用したBERTモデルの強力さがわかったであろう

応用として、例えば、最近よく見る自動レスポンスチャットのように、ある質問に対する回答を導き出すこともできる
- Amazon Alexaや、Goole アシスタントのようなことも、なんとなくイメージできるようになったのではないだろうか

# 課題

次のようなプログラムを作成し実行しなさい

- 上記の記事分類の保存済みモデルをロードする
  - 保存済みモデルは、ダウンロードして自分で別途保存し、この別途保存したファイルをcontext内にドロップして配置してからプログラムを実行するとよい
  - 保存済みモデルをどのように与えるかは各自で考えること
  - 例えば、Google Driveをマウントしてもよい
- 3つの異なるトピックのランダムな記事を選択し、実際に分類が正解するか確認しなさい
- 2つの「自分で考えた記事」を用いて分類し、その分類について100字未満で簡単に考察しなさい